# **환경 설정**

In [ ]:
! pip install langchain_community tiktoken langchain-openai chromadb langchain langgraph tavily-python

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")

# **벡터 DB 구축**

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.docment_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# 크롤링할 블로그의 url 정의
urls = [
    "https://google.github.io/styleguide/pyguide.html",
    "https://google.github.io/styleguide/javaguide.html",
    "https://google.github.io/styleguide/jsguide.html",
]

# WebBaseLoader를 사용하여 주어진 URL 목록에서 문서 크롤링
docs = [WebBaseLoader(url).load() for url in urls]
# docs에서 sublist 이름으로 하나씩 추출
# -> sublist를 다시 item으로 추출
# 맨 왼쪽의 최종 반환값
docs_list = [item for sublist in docs for item in sublist]

# 지정한 크기만큼 텍스트를 분할하는 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)

# 문서 분할
doc_splits = text_splitter.split_documents(docs_list)

# Chroma 벡터 저장소에 문서의 분할된 조각 저장
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=OpenAIEnbeddings(),
)

# 벡터 저장소에서 검색을 수행할 수 있는 검색기 생성
retriever = vectorstore.as_retriever()